# Functional API


Not all neural network models are simply sequential. Some may have complex topologies. Some may have multiple inputs and/or multiple outputs. For example, a Wide & Deep neural network (see paper) connects all or part of the inputs directly to the output layer.
Let’s build such a neural network to tackle the California housing problem:

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)



In [5]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_], outputs=[output])


In [6]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]         

In [7]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 0s 872us/step - loss: 1.3035 - val_loss: 0.7101
Epoch 2/20
363/363 [==============================] - 0s 551us/step - loss: 0.6619 - val_loss: 0.6456
Epoch 3/20
363/363 [==============================] - 0s 531us/step - loss: 0.6156 - val_loss: 0.6133
Epoch 4/20
363/363 [==============================] - 0s 526us/step - loss: 0.5873 - val_loss: 0.5928
Epoch 5/20
363/363 [==============================] - 0s 523us/step - loss: 0.5664 - val_loss: 0.5708
Epoch 6/20
363/363 [==============================] - 0s 538us/step - loss: 0.5470 - val_loss: 0.5567
Epoch 7/20
363/363 [==============================] - 0s 533us/step - loss: 0.5324 - val_loss: 0.5430
Epoch 8/20
363/363 [==============================] - 0s 517us/step - loss: 0.5193 - val_loss: 0.5318
Epoch 9/20
363/363 [==============================] - 0s 518us/step - loss: 0.5088 - val_loss: 0.5219
Epoch 10/20
363/363 [==============================] - 0s 520us/step - loss: 0.499

What if you want to send different subsets of input features through the wide or deep paths? We will send 5 features (features 0 to 4), and 6 through the deep path (features 2 to 7). Note that 3 features will go through both (features 2, 3 and 4).


In [8]:
np.random.seed(42)
tf.random.set_seed(42)

In [9]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [10]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [11]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [12]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 0s 771us/step - loss: 1.8248 - val_loss: 0.7923
Epoch 2/20
363/363 [==============================] - 0s 567us/step - loss: 0.7055 - val_loss: 0.6450
Epoch 3/20
363/363 [==============================] - 0s 540us/step - loss: 0.6258 - val_loss: 0.6057
Epoch 4/20
363/363 [==============================] - 0s 545us/step - loss: 0.5910 - val_loss: 0.5840
Epoch 5/20
363/363 [==============================] - 0s 544us/step - loss: 0.5690 - val_loss: 0.5665
Epoch 6/20
363/363 [==============================] - 0s 539us/step - loss: 0.5508 - val_loss: 0.5542
Epoch 7/20
363/363 [==============================] - 0s 536us/step - loss: 0.5373 - val_loss: 0.5430
Epoch 8/20
363/363 [==============================] - 0s 538us/step - loss: 0.5256 - val_loss: 0.5343
Epoch 9/20
363/363 [==============================] - 0s 536us/step - loss: 0.5168 - val_loss: 0.5270
Epoch 10/20
363/363 [==============================] - 0s 543us/step - loss: 0.509

Adding an auxiliary output for regularization:

In [13]:
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

In [15]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

In [16]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 0s 919us/step - loss: 2.1377 - main_output_loss: 1.9300 - aux_output_loss: 4.0068 - val_loss: 1.0631 - val_main_output_loss: 0.8288 - val_aux_output_loss: 3.1725
Epoch 2/20
363/363 [==============================] - 0s 618us/step - loss: 0.9125 - main_output_loss: 0.7249 - aux_output_loss: 2.6008 - val_loss: 0.8148 - val_main_output_loss: 0.6557 - val_aux_output_loss: 2.2465
Epoch 3/20
363/363 [==============================] - 0s 607us/step - loss: 0.7656 - main_output_loss: 0.6357 - aux_output_loss: 1.9353 - val_loss: 0.7374 - val_main_output_loss: 0.6142 - val_aux_output_loss: 1.8463
Epoch 4/20
363/363 [==============================] - 0s 618us/step - loss: 0.7043 - main_output_loss: 0.5994 - aux_output_loss: 1.6482 - val_loss: 0.6974 - val_main_output_loss: 0.5923 - val_aux_output_loss: 1.6432
Epoch 5/20
363/363 [==============================] - 0s 623us/step - loss: 0.6687 - main_output_loss: 0.5769 - aux_output_loss: 1.4948 

In [17]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

162/162 [==============================] - 0s 454us/step - loss: 0.5287 - main_output_loss: 0.4825 - aux_output_loss: 0.9444
